## Imports

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, BayesianRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime

DATEPARSE = "%Y-%m-%d %H:%M:%S"

# Read data
### Add data into /taxi_data/
### Github stores a sample file of this. Download the rest and do the same.

In [32]:
def add_zero_padding(number):
    if len(str(number)) == 1: return "0" + str(number)
    else: return number
def format_df(DF, pickup_date_column, dropof_date_column):
    for i, x in DF.iterrows():
        DF.at[i,"month"] = 1
        start = datetime.strptime(x[pickup_date_column], DATEPARSE).time()
        stop = datetime.strptime(x[dropof_date_column], DATEPARSE).time()

        DF.at[i, pickup_date_column] = (start.hour * 60 + start.minute)
        DF.at[i, dropof_date_column] = (stop.hour * 60 + stop.minute)
        DF.at[i, "travel_time"] = (stop.hour * 60 + stop.minute) - (start.hour * 60 + start.minute)
    return DF

In [34]:
#GREEN
header = True
for x in range(1, 13): # this takes 14 min
    try:
        dfGreen = pd.read_csv("taxi_data\green_tripdata_2019-" + str(add_zero_padding(x)) + ".csv", usecols=["PULocationID", "DOLocationID", "lpep_pickup_datetime", "lpep_dropoff_datetime"])
        dfGreen = format_df(dfGreen, "lpep_pickup_datetime", "lpep_dropoff_datetime")
        dfGreen.to_csv("taxi_data\green_tripdata_2019_concat.csv", mode='a+', header=header, index=False)
        header=False
        print("GREEN: " + str(x))
    except Exception as f:
        print(f)

"""
# YELLOW
header = True
for x in range(1, 13):
    try:
        dfYellow = pd.read_csv("taxi_data\yellow_tripdata_2019-" + str(add_zero_padding(x)) + ".csv")
        dfYellow = format_df(dfYellow, "tpep_pickup_datetime", "tpep_dropoff_datetime")
        dfYellow.to_csv("taxi_data\yellow_tripdata_2019_concat.csv", mode='a+', header=header, index=False)
        header=False
        print("YELLOW: " + str(x))
    except Exception as f:
        print(f)
"""

# Train

In [31]:
dfGreen = pd.read_csv("taxi_data\green_tripdata_2019_concat.csv", chunksize=100000)

model = KNeighborsClassifier(3)

first = True
for chunk in dfGreen:
    print(chunk)
    X_green = chunk[["PULocationID", "DOLocationID","lpep_pickup_datetime", "month"]] # input cols
    y_green = chunk[["travel_time"]] # output cols
    print(X_green[0])
    

    X_train, X_test, y_train, y_test = train_test_split(X_green, y_green, test_size = 0.25)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
    print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))
    
    break

       lpep_pickup_datetime  lpep_dropoff_datetime  PULocationID  \
0                       917                    918           264   
1                        10                     16            97   
2                        27                     31            49   
3                        46                     64           189   
4                        19                     39            82   
...                     ...                    ...           ...   
99995                   596                    601           130   
99996                   552                    592           218   
99997                   542                    577           139   
99998                   595                    604            72   
99999                   599                    624            89   

       DOLocationID  month  travel_time  
0               264    1.0         -1.0  
1                49    1.0         -6.0  
2               189    1.0         -4.0  
3              

KeyError: 0